In [1]:
# 関数定義

import boto3

def make_question_and_ground_truth(
    region: str,
    model_id: str = "us.meta.llama4-maverick-17b-instruct-v1:0"
):
    runtime_client = boto3.client("bedrock-runtime", region_name=region)
    prompt = """東京都への観光客がしそうな質問とそれに対する模範回答を考えてください。ただし質問は<question></question>、模範回答は<answer></answer>で囲んでください。"""
    try:
        response = runtime_client.converse(
            modelId=model_id,
            messages=[{"role": "user", "content": [{"text": prompt}]}]
        )
    except Exception as e:
        print(f"予期せぬエラーが発生しました: {e}")
        raise
    
    message = response["output"].get("message")
    if message:
        res = message["content"][0].get("text")
        if res:
            return res
        else:
            raise ValueError(f"{model_id}からのレスポンスに text が含まれませんでした: {message}")
    else:
        raise ValueError(f"{model_id} からのレスポンスに message が含まれませんでした: {response}")

In [2]:
import os
from dotenv import load_dotenv

load_dotenv("./.env")

REGION = os.environ.get("REGION")
if REGION:
    qa = make_question_and_ground_truth(region=REGION)
else:
    raise ValueError(f"環境変数を読み取れませんでした。\nREGION: {REGION}")



In [3]:
from pprint import pprint

pprint(qa)

('<question>東京の代表的な観光スポットはどこですか？</question>\n'
 '<answer>東京の代表的な観光地には、浅草の雷門や渋谷のスクランブル交差点、東京タワーや東京スカイツリーなどがあります。特に浅草の仲見世通りや渋谷のハチ公像周辺は、国内外の観光客でにぎわっています。</answer>\n'
 '\n'
 '<question>東京の交通手段は何が便利ですか？</question>\n'
 '<answer>東京の交通網は非常に発達しており、電車や地下鉄、バスが主要な交通手段です。特にJRや東京メトロの路線が広範囲にわたっており、主要な観光地へのアクセスが容易です。また、SuicaやPASMOなどのICカードを利用すると、スムーズに移動できます。</answer>\n'
 '\n'
 '<question>東京で人気のグルメはなんですか？</question>\n'
 '<answer>東京には多様なグルメが楽しめますが、特に人気なのは寿司やラーメン、とんかつです。築地市場や銀座の高級寿司店、浅草の老舗ラーメン店など、さまざまな飲食店が観光客に人気です。また、居酒屋や焼き鳥屋も気軽に楽しめます。</answer>\n'
 '\n'
 '<question>東京の買い物スポットはどこですか？</question>\n'
 '<answer>東京にはさまざまな買い物スポットがありますが、特に人気なのは銀座や渋谷、新宿です。銀座では高級ブランドショップが立ち並び、渋谷の109や渋谷スクランブルスクエアでは若者向けのファッションアイテムが豊富です。新宿の伊勢丹や高島屋も人気のデパートです。</answer>\n'
 '\n'
 '<question>東京で訪れるべき神社や寺院はどこですか？</question>\n'
 '<answer>東京には歴史ある神社や寺院が数多くあります。明治神宮や靖国神社、浅草寺などが特に有名です。明治神宮は明治天皇を祀る神社で、初詣の時期には多くの参拝客で賑わいます。浅草寺は東京都内最古の寺院で、浅草のシンボルである雷門が有名です。</answer>')


In [4]:
import re

pattern = r"<question>(.*?)</question>\n<answer>(.*?)</answer>"
matches = re.findall(pattern, qa, re.DOTALL)

In [5]:
qa = {
    "question": [m[0] for m in matches],
    "answer": [m[1] for m in matches]
}

In [6]:
qa

{'question': ['東京の代表的な観光スポットはどこですか？',
  '東京の交通手段は何が便利ですか？',
  '東京で人気のグルメはなんですか？',
  '東京の買い物スポットはどこですか？',
  '東京で訪れるべき神社や寺院はどこですか？'],
 'answer': ['東京の代表的な観光地には、浅草の雷門や渋谷のスクランブル交差点、東京タワーや東京スカイツリーなどがあります。特に浅草の仲見世通りや渋谷のハチ公像周辺は、国内外の観光客でにぎわっています。',
  '東京の交通網は非常に発達しており、電車や地下鉄、バスが主要な交通手段です。特にJRや東京メトロの路線が広範囲にわたっており、主要な観光地へのアクセスが容易です。また、SuicaやPASMOなどのICカードを利用すると、スムーズに移動できます。',
  '東京には多様なグルメが楽しめますが、特に人気なのは寿司やラーメン、とんかつです。築地市場や銀座の高級寿司店、浅草の老舗ラーメン店など、さまざまな飲食店が観光客に人気です。また、居酒屋や焼き鳥屋も気軽に楽しめます。',
  '東京にはさまざまな買い物スポットがありますが、特に人気なのは銀座や渋谷、新宿です。銀座では高級ブランドショップが立ち並び、渋谷の109や渋谷スクランブルスクエアでは若者向けのファッションアイテムが豊富です。新宿の伊勢丹や高島屋も人気のデパートです。',
  '東京には歴史ある神社や寺院が数多くあります。明治神宮や靖国神社、浅草寺などが特に有名です。明治神宮は明治天皇を祀る神社で、初詣の時期には多くの参拝客で賑わいます。浅草寺は東京都内最古の寺院で、浅草のシンボルである雷門が有名です。']}

In [7]:
from backend import call_rag

knowledge_base_id = os.environ.get("KNOWLEDGE_BASE_ID")
responses = {
    "rag_answer": [],
    "retrieved_context": []
}
if knowledge_base_id:
    for question in qa["question"]:
        res = call_rag(question=question, knowledge_base_id=knowledge_base_id, region=REGION)
        responses["rag_answer"].append(res["answer"])
        responses["retrieved_context"].append(res["context"])
else:
    print("環境変数を読み取れませんでした。")

In [8]:
responses

{'rag_answer': ['わかりません',
  '東京の交通手段としては、東京モノレールや京急電鉄、リムジンバス、タクシーが便利です。',
  'わかりません',
  '東京の買い物スポットは、上記の情報から、下記のように考えることができまど。\n\n東京都内では、上記のように、大型のシップストアがある。\n\nこれらの情報から、セレクトしたいときは、上記のスポットを参考にしてください。',
  'おさかなところです。'],
 'retrieved_context': ['また、主に観光目的で東京湾から隅田川にかけての航路が整備されているが、旧東京市内の東部に広がっていた河川や運河の水路網は戦後復興におけるがれき処理などで大半が埋め立てられ、東京都内での内水路利用は大阪と比較すると非常に限定的である。 ## 情報・メディア \\[編集\\]\n===\n?title=%E6%9D%B1%E4%BA%AC%E9%83%BD%E6%B8%8B%E8%B0%B7%E5%85%AC%E5%9C%92%E9%80%9A%E3%82%8A%E3%82%AE%E3%83%A3%E3%83%A9%E3%83%AA%E3%83%BC&action=edit&redlink=1)、[東京文化会館](https://ja.wikipedia.org/wiki/%E6%9D%B1%E4%BA%AC%E6%96%87%E5%8C%96%E4%BC%9A%E9%A4%A8)、[東京芸術劇場](https://ja.wikipedia.org/wiki/%E6%9D%B1%E4%BA%AC%E8%8A%B8%E8%A1%93%E5%8A%87%E5%A0%B4)などがある[\\[79\\]。 ### 観光 \\[編集\\] ![]() 正福寺 地蔵堂（東村山市 国宝）（2010年6月11日撮影\n===\n東京都が運営する文化施設には、東京都江戸東京博物館](https://ja.wikipedia.org/wiki/%E6%9D%B1%E4%BA%AC%E9%83%BD%E6%B1%9F%E6%88%B8%E6%9D%B1%E4%BA%AC%E5%8D%9A%E7%89%A9%E9%A4%A8)、[江戸東京たてもの園](https://ja.wikipedia.org/wiki/%E6

In [9]:
import pandas as pd

merged = qa | responses
df = pd.DataFrame(merged)

In [10]:
df

,question,answer,rag_answer,retrieved_context
0,東京の代表的な観光スポットはどこですか？,東京の代表的な観光地には、浅草の雷門や渋谷のスクランブル交差点、東京タワーや東京スカイツリー...,わかりません,また、主に観光目的で東京湾から隅田川にかけての航路が整備されているが、旧東京市内の東部に広が...
1,東京の交通手段は何が便利ですか？,東京の交通網は非常に発達しており、電車や地下鉄、バスが主要な交通手段です。特にJRや東京メト...,東京の交通手段としては、東京モノレールや京急電鉄、リムジンバス、タクシーが便利です。,都心部との交通手段として[東京モノレール](https://ja.wikipedia.org...
2,東京で人気のグルメはなんですか？,東京には多様なグルメが楽しめますが、特に人気なのは寿司やラーメン、とんかつです。築地市場や銀...,わかりません,伴う物価高等で景気が冷え込んだ1975年以降は鈍化し、1980年には初めて人口が減少した。1...
3,東京の買い物スポットはどこですか？,東京にはさまざまな買い物スポットがありますが、特に人気なのは銀座や渋谷、新宿です。銀座では高...,東京の買い物スポットは、上記の情報から、下記のように考えることができまど。\n\n東京都内で...,16. **^](https://ja.wikipedia.org/wiki/%E6%9D%...
4,東京で訪れるべき神社や寺院はどこですか？,東京には歴史ある神社や寺院が数多くあります。明治神宮や靖国神社、浅草寺などが特に有名です。明...,おさかなところです。,[新島空港](https://ja.wikipedia.org/wiki/%E6%96%B0...


In [11]:
df.to_pickle("./eval_data/eval_data.pkl")

In [33]:
from ragas import evaluate
from datasets import Dataset
from ragas.metrics import answer_relevancy, answer_similarity, context_recall, faithfulness
from langchain_aws import ChatBedrockConverse, BedrockEmbeddings

In [37]:
new_df = df.rename(columns={"answer": "reference", "rag_answer": "response", "retrieved_context": "retrieved_contexts"})
new_df["retrieved_contexts"] = new_df["retrieved_contexts"].apply(lambda x: [x])
new_df

,question,reference,response,retrieved_contexts
0,東京の代表的な観光スポットはどこですか？,東京の代表的な観光地には、浅草の雷門や渋谷のスクランブル交差点、東京タワーや東京スカイツリー...,わかりません,[また、主に観光目的で東京湾から隅田川にかけての航路が整備されているが、旧東京市内の東部に広...
1,東京の交通手段は何が便利ですか？,東京の交通網は非常に発達しており、電車や地下鉄、バスが主要な交通手段です。特にJRや東京メト...,東京の交通手段としては、東京モノレールや京急電鉄、リムジンバス、タクシーが便利です。,[都心部との交通手段として[東京モノレール](https://ja.wikipedia.or...
2,東京で人気のグルメはなんですか？,東京には多様なグルメが楽しめますが、特に人気なのは寿司やラーメン、とんかつです。築地市場や銀...,わかりません,[伴う物価高等で景気が冷え込んだ1975年以降は鈍化し、1980年には初めて人口が減少した。...
3,東京の買い物スポットはどこですか？,東京にはさまざまな買い物スポットがありますが、特に人気なのは銀座や渋谷、新宿です。銀座では高...,東京の買い物スポットは、上記の情報から、下記のように考えることができまど。\n\n東京都内で...,[16. **^](https://ja.wikipedia.org/wiki/%E6%9D...
4,東京で訪れるべき神社や寺院はどこですか？,東京には歴史ある神社や寺院が数多くあります。明治神宮や靖国神社、浅草寺などが特に有名です。明...,おさかなところです。,[[新島空港](https://ja.wikipedia.org/wiki/%E6%96%B...


In [38]:
dataset = Dataset.from_pandas(new_df)
metrics = [answer_relevancy, answer_similarity, context_recall, faithfulness]

runtime_client = boto3.client("bedrock-runtime", region_name=REGION)
llm = ChatBedrockConverse(
    client=runtime_client,
    model="us.meta.llama4-maverick-17b-instruct-v1:0",
    temperature=0.0
)
embeddings = BedrockEmbeddings(
    client=runtime_client,
    model_id="amazon.titan-embed-text-v2:0"
)

In [39]:
result = evaluate(
    dataset=dataset,
    metrics=metrics,
    llm=llm,
    embeddings=embeddings
)

Evaluating:  85%|████████▌ | 17/20 [00:08<00:01,  2.76it/s]Exception raised in Job[3]: OutputParserException(Failed to parse StringIO from completion {"statements": ["The representative tourist spots in Tokyo are not known."]}. Got: 1 validation error for StringIO
text
  Field required [type=missing, input_value={'statements': ['The repr... Tokyo are not known.']}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE )
Evaluating: 100%|██████████| 20/20 [00:11<00:00,  1.79it/s]


In [40]:
result

{'answer_relevancy': 0.1818, 'semantic_similarity': 0.3740, 'context_recall': 0.2000, 'faithfulness': 0.3542}